In [ ]:
!pip install -q langchain_openai openai faiss-cpu pypdf pandas langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
AZURE_API_KEY=userdata.get('AZURE_API_KEY')
AZURE_ENDPOINT=userdata.get('AZURE_ENDPOINT')

In [ ]:
from langchain.chat_models import AzureChatOpenAI
import random
import json

# Azure LLM Initialization
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    azure_deployment="gpt-4o",
    api_version="2023-09-01-preview",
    api_key=AZURE_API_KEY,
    temperature=0
)


In [ ]:
import json
import re

def safe_json_parse(response):
    """Cleans and extracts JSON from the response."""
    if not response or response.strip() == "":
        print("Error: Empty response from LLM!")
        return None

    # Remove ```json and ``` if present
    response = response.strip().strip("```json").strip("```")

    # Extract only the JSON part using regex (ensures robustness)
    match = re.search(r'\[.*\]', response, re.DOTALL)
    json_text = match.group(0) if match else response

    try:
        return json.loads(json_text)  # Parse the cleaned JSON
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        return None

def generate_hospital_data():
    prompt = "Generate a JSON list of 5 fake hospitals. Return valid JSON only."
    response = llm([HumanMessage(content=prompt)]).content  # Get response

    print("Raw LLM Response:", response)  # Debugging
    return safe_json_parse(response)

hospitals = generate_hospital_data()
if hospitals:
    print(json.dumps(hospitals, indent=2))  # Pretty-print the JSON
else:
    print("Failed to generate valid hospital data.")


Raw LLM Response: ```json
[
  {
    "name": "Riverside General Hospital",
    "address": "123 Elm Street, Springfield, IL 62701",
    "phone": "(217) 555-1234",
    "website": "http://www.riversidegeneral.com",
    "specialties": ["Cardiology", "Orthopedics", "Pediatrics"]
  },
  {
    "name": "Pinecrest Medical Center",
    "address": "456 Oak Avenue, Denver, CO 80203",
    "phone": "(303) 555-5678",
    "website": "http://www.pinecrestmed.com",
    "specialties": ["Neurology", "Oncology", "Emergency Medicine"]
  },
  {
    "name": "Lakeside Health Institute",
    "address": "789 Maple Drive, Austin, TX 73301",
    "phone": "(512) 555-9012",
    "website": "http://www.lakesidehealth.org",
    "specialties": ["Dermatology", "Gastroenterology", "Rehabilitation"]
  },
  {
    "name": "Summit View Hospital",
    "address": "321 Birch Lane, Seattle, WA 98101",
    "phone": "(206) 555-3456",
    "website": "http://www.summitviewhospital.com",
    "specialties": ["Pulmonology", "Endocrinolog

In [ ]:
hospitals

[{'name': 'Riverside General Hospital',
  'address': '123 Elm Street, Springfield, IL 62701',
  'phone': '(217) 555-1234',
  'website': 'http://www.riversidegeneral.com',
  'specialties': ['Cardiology', 'Orthopedics', 'Pediatrics']},
 {'name': 'Pinecrest Medical Center',
  'address': '456 Oak Avenue, Denver, CO 80203',
  'phone': '(303) 555-5678',
  'website': 'http://www.pinecrestmed.com',
  'specialties': ['Neurology', 'Oncology', 'Emergency Medicine']},
 {'name': 'Lakeside Health Institute',
  'address': '789 Maple Drive, Austin, TX 73301',
  'phone': '(512) 555-9012',
  'website': 'http://www.lakesidehealth.org',
  'specialties': ['Dermatology', 'Gastroenterology', 'Rehabilitation']},
 {'name': 'Summit View Hospital',
  'address': '321 Birch Lane, Seattle, WA 98101',
  'phone': '(206) 555-3456',
  'website': 'http://www.summitviewhospital.com',
  'specialties': ['Pulmonology', 'Endocrinology', 'General Surgery']},
 {'name': 'Cedar Valley Medical Center',
  'address': '654 Pine Stre

In [ ]:
with open("hospitals.json", "w", encoding="utf-8") as f:
    json.dump(hospitals, f, indent=4)

print("Data saved to hospitals.json successfully!")

Data saved to hospitals.json successfully!


In [ ]:
from langchain.schema import HumanMessage
import json
import re

def clean_json_response(response):
    """Extracts JSON from a response with possible extra text."""
    match = re.search(r'\[.*\]', response, re.DOTALL)  # Extract JSON inside square brackets
    if match:
        return match.group(0)  # Extract JSON content
    return response  # If no match, return as-is (may still fail)

def generate_hospital_data():
    prompt = "Generate a JSON list of 5 fake hospitals. Return valid JSON only."
    response = llm([HumanMessage(content=prompt)]).content

    print("Raw LLM Response:", response)  # Debugging

    cleaned_response = clean_json_response(response)
    try:
        return json.loads(cleaned_response)
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        return None

hospitals = generate_hospital_data()

# Agent 2: Doctor Data Generator
def generate_doctor_data(hospitals):
    prompt = f"Generate a JSON list of 10 fake doctors with fields: doctor_id, name, specialization, experience, availability, hospital_id, fees. Assign them to one of these hospitals: {json.dumps([h['hospital_id'] for h in hospitals])}."
    response = llm([HumanMessage(content=prompt)]).content
    return json.loads(response)

# Agent 3: Patient Data Generator
def generate_patient_data():
    prompt = "Generate a JSON list of 10 fake patients with fields: patient_id, name, age, gender, symptoms, department_suggested, contact_number."
    response = llm([HumanMessage(content=prompt)]).content
    return json.loads(response)

# Agent 4: Appointment Data Generator
def generate_appointments(doctors, patients):
    prompt = f"Generate a JSON list of 10 fake appointments with fields: appointment_id, patient_id, doctor_id, hospital_id, date, time_slot, status. Use these patient IDs: {json.dumps([p['patient_id'] for p in patients])} and doctor IDs: {json.dumps([d['doctor_id'] for d in doctors])}."
    response = llm([HumanMessage(content=prompt)]).content
    return json.loads(response)


Raw LLM Response: ```json
[
  {
    "name": "Riverside General Hospital",
    "address": "123 Elm Street, Springfield, IL 62701",
    "phone": "(217) 555-1234",
    "website": "www.riversidegeneral.com",
    "specialties": ["Cardiology", "Orthopedics", "Pediatrics"]
  },
  {
    "name": "Summit Valley Medical Center",
    "address": "456 Oak Avenue, Denver, CO 80203",
    "phone": "(303) 555-5678",
    "website": "www.summitvalleymed.com",
    "specialties": ["Neurology", "Oncology", "Emergency Medicine"]
  },
  {
    "name": "Lakeside Community Hospital",
    "address": "789 Pine Road, Madison, WI 53703",
    "phone": "(608) 555-9012",
    "website": "www.lakesidehospital.org",
    "specialties": ["Dermatology", "Gastroenterology", "General Surgery"]
  },
  {
    "name": "Cedar Hills Regional Medical Center",
    "address": "321 Maple Lane, Austin, TX 78701",
    "phone": "(512) 555-3456",
    "website": "www.cedarhillsmed.com",
    "specialties": ["Psychiatry", "Rehabilitation", "Int

In [ ]:
hospitals = generate_hospital_data()
doctors = generate_doctor_data(hospitals)
patients = generate_patient_data()
appointments = generate_appointments(doctors, patients)

    # Save generated data to JSON files
with open("hospitals.json", "w") as f:
  json.dump(hospitals, f, indent=4)

with open("doctors.json", "w") as f:
  json.dump(doctors, f, indent=4)

with open("patients.json", "w") as f:
  json.dump(patients, f, indent=4)

with open("appointments.json", "w") as f:
  json.dump(appointments, f, indent=4)

print("Fake data generated and saved successfully!")

LLM Raw Response:
 ```json
[
  {
    "hospital_id": "HSP001",
    "name": "Sunrise Medical Center",
    "address": "123 Elm Street, Springfield, IL, 62701",
    "contact_number": "+1-217-555-1234",
    "email": "info@sunrisemedical.com",
    "departments": ["Cardiology", "Orthopedics", "Pediatrics", "Neurology", "Emergency"],
    "opening_hours": "24/7",
    "rating": 4.5
  },
  {
    "hospital_id": "HSP002",
    "name": "Green Valley Hospital",
    "address": "456 Oak Avenue, Green Valley, CA, 90210",
    "contact_number": "+1-310-555-5678",
    "email": "contact@greenvalleyhospital.com",
    "departments": ["General Surgery", "Dermatology", "Oncology", "Radiology", "Maternity"],
    "opening_hours": "Mon-Fri: 8 AM - 8 PM, Sat-Sun: 9 AM - 5 PM",
    "rating": 4.2
  },
  {
    "hospital_id": "HSP003",
    "name": "Lakeside Health Center",
    "address": "789 Lakeview Road, Lakeside, TX, 75001",
    "contact_number": "+1-972-555-9012",
    "email": "support@lakesidehealth.com",
    "dep

TypeError: 'NoneType' object is not iterable

In [ ]:

import json
import random
import time



def generate_hospital_data():
    prompt = """
    Generate a JSON array of 10 hospitals with the following fields:
    - hospital_id (unique)
    - name
    - address
    - contact_number
    - email
    - departments (list of 5-7 specialties)
    - opening_hours
    - rating (1-5 float)

    Ensure the response is a valid JSON array.
    """
    response = llm.invoke(prompt).content.strip()
    print(response)
    return response

def generate_doctor_data(hospitals):
    prompt = """
    Generate a JSON array of 60 doctors, each with:
    - doctor_id (unique)
    - name
    - specialization (choose from hospital departments)
    - hospital_id (link to an existing hospital)
    - experience_years (1-30)
    - contact_number
    - email

    Ensure the response is a valid JSON array.
    """
    response = llm.invoke(prompt).content.strip()
    print(response)
    return response

def generate_patient_data():
    prompt = """
    Generate a JSON array of 200 patients with:
    - patient_id (unique)
    - name
    - age (10-90)
    - gender (Male, Female, Other)
    - contact_number
    - email
    - medical_history (list of 2-4 conditions)

    Ensure the response is a valid JSON array.
    """
    response = llm.invoke(prompt).content.strip()
    print(response)
    return response

def generate_appointments(doctors, patients):
    prompt = """
    Generate a JSON array of 100 appointments with:
    - appointment_id (unique)
    - patient_id (link to an existing patient)
    - doctor_id (link to an existing doctor)
    - department (based on doctor's specialization)
    - date (YYYY-MM-DD format, random future date)
    - time (HH:MM format)
    - status (Scheduled, Completed, Cancelled)

    Ensure the response is a valid JSON array.
    """
    response = llm.invoke(prompt).content.strip()
    print(response)
    return response

# Run data generation
hospitals = generate_hospital_data()
time.sleep(1)  # Prevent rate limits
doctors = generate_doctor_data(hospitals)
time.sleep(1)
patients = generate_patient_data()
time.sleep(1)
appointments = generate_appointments(doctors, patients)

# Save data to JSON files
datasets = {"hospitals": hospitals, "doctors": doctors, "patients": patients, "appointments": appointments}

for name, data in datasets.items():
    with open(f"{name}.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

print("Data generation complete! JSON files saved.")


```json
[
  {
    "hospital_id": 1,
    "name": "Green Valley Medical Center",
    "address": "123 Elm Street, Springfield, IL",
    "contact_number": "+1-555-123-4567",
    "email": "info@greenvalleymedical.com",
    "departments": ["Cardiology", "Neurology", "Pediatrics", "Orthopedics", "Dermatology", "Oncology"],
    "opening_hours": "Mon-Fri: 8 AM - 8 PM, Sat-Sun: 9 AM - 5 PM",
    "rating": 4.5
  },
  {
    "hospital_id": 2,
    "name": "Sunrise General Hospital",
    "address": "456 Oak Avenue, Denver, CO",
    "contact_number": "+1-555-234-5678",
    "email": "contact@sunrisehospital.com",
    "departments": ["Emergency", "Gynecology", "Urology", "Radiology", "Endocrinology", "Psychiatry"],
    "opening_hours": "24/7",
    "rating": 4.2
  },
  {
    "hospital_id": 3,
    "name": "Lakeside Community Hospital",
    "address": "789 Pine Road, Austin, TX",
    "contact_number": "+1-555-345-6789",
    "email": "support@lakesidehospital.com",
    "departments": ["Pediatrics", "Cardiol

In [ ]:
def safe_json_parse(response):
    """Cleans and extracts JSON from the response."""
    if not response or response.strip() == "":
        print("Error: Empty response from LLM!")
        return None

    # Remove ```json and ``` if present
    response = response.strip().strip("```json").strip("```")

    # Extract only the JSON part using regex (ensures robustness)
    match = re.search(r'\[.*\]', response, re.DOTALL)
    json_text = match.group(0) if match else response

    try:
        return json.loads(json_text)  # Parse the cleaned JSON
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        return None

In [ ]:
hospitals=safe_json_parse(hospitals)
patients=safe_json_parse(patients)
appointments=safe_json_parse(appointments)
doctors=safe_json_parse(doctors)

In [ ]:
with open("hospitals.json", "w", encoding="utf-8") as f:
    json.dump(hospitals, f, indent=4)

print("Data saved to hospitals.json successfully!")

Data saved to hospitals.json successfully!


In [ ]:
with open("doctors.json", "w", encoding="utf-8") as f:
    json.dump(doctors, f, indent=4)

print("Data saved to doctors.json successfully!")

Data saved to doctors.json successfully!


In [ ]:
with open("appointments.json", "w", encoding="utf-8") as f:
    json.dump(appointments, f, indent=4)

print("Data saved to appointments.json successfully!")

Data saved to appointments.json successfully!


In [ ]:
with open("patients.json", "w", encoding="utf-8") as f:
    json.dump(patients, f, indent=4)

print("Data saved to patients.json successfully!")

Data saved to patients.json successfully!


In [ ]:
import json
import random

# Load existing data
with open("patients.json", "r", encoding="utf-8") as f:
    patients = json.load(f)

with open("doctors.json", "r", encoding="utf-8") as f:
    doctors = json.load(f)

with open("hospitals.json", "r", encoding="utf-8") as f:
    hospitals = json.load(f)

# Extract valid IDs
patient_ids = [p["patient_id"] for p in patients]
doctor_data = {str(d["doctor_id"]): d for d in doctors}  # Convert doctor_id to string for consistency
hospital_ids = {str(h["hospital_id"]) for h in hospitals}  # Ensure hospital_id is valid

# Fix doctors' hospital IDs
for doctor in doctors:
    if str(doctor["hospital_id"]) not in hospital_ids:
        doctor["hospital_id"] = random.choice(list(hospital_ids))  # Assign a valid hospital

# Generate new appointments with correct links
corrected_appointments = []
for i in range(10):  # Generate 10 appointments
    selected_doctor_id = random.choice(list(doctor_data.keys()))
    selected_doctor = doctor_data[selected_doctor_id]

    appointment = {
        "appointment_id": f"APT{i+1:03d}",
        "patient_id": random.choice(patient_ids),  # Ensure valid patient
        "doctor_id": selected_doctor_id,  # Ensure valid doctor
        "department": selected_doctor["specialization"],  # Ensure correct department
        "date": f"2025-03-{random.randint(11, 31)}",  # Random future date
        "time": f"{random.randint(9, 17)}:{random.choice(['00', '30'])}",  # Random time slot
        "status": random.choice(["Scheduled", "Completed", "Cancelled"]),
    }
    corrected_appointments.append(appointment)

# Save corrected doctors.json
with open("doctors.json", "w", encoding="utf-8") as f:
    json.dump(doctors, f, indent=4)

# Save corrected appointments
with open("appointments.json", "w", encoding="utf-8") as f:
    json.dump(corrected_appointments, f, indent=4)

print("✅ Appointments updated with valid doctor specializations and hospital IDs.")


✅ Appointments updated with valid doctor specializations and hospital IDs.


In [ ]:
import json

# List of JSON file names
file_names = ["patients.json", "doctors.json", "hospitals.json", "appointments.json"]

# Dictionary to store features for each file
file_features = {}

# Extract features from each file
for file in file_names:
    try:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

            # Ensure it's a list and contains at least one record
            if isinstance(data, list) and len(data) > 0:
                file_features[file] = list(data[0].keys())
            else:
                file_features[file] = "No data found or invalid format"

    except Exception as e:
        file_features[file] = f"Error reading file: {e}"

# Print the extracted features
for file, features in file_features.items():
    print(f"\n🔹 Features in {file}:")
    print(features)



🔹 Features in patients.json:
['patient_id', 'name', 'age', 'gender', 'contact_number', 'email', 'medical_history']

🔹 Features in doctors.json:
['doctor_id', 'name', 'specialization', 'hospital_id', 'experience_years', 'contact_number', 'email']

🔹 Features in hospitals.json:
['hospital_id', 'name', 'address', 'contact_number', 'email', 'departments', 'opening_hours', 'rating']

🔹 Features in appointments.json:
['appointment_id', 'patient_id', 'doctor_id', 'department', 'date', 'time', 'status']


In [ ]:
!pip install sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
import sqlite3
import json

# Load JSON data
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as file:
        return json.load(file)

# Connect to SQLite database
conn = sqlite3.connect("hospital_db.sqlite3")
cursor = conn.cursor()

# Drop tables if they exist (for clean re-run)
cursor.executescript("""
DROP TABLE IF EXISTS Appointments;
DROP TABLE IF EXISTS Doctors;
DROP TABLE IF EXISTS Patients;
DROP TABLE IF EXISTS Hospitals;
""")

# Create tables
cursor.executescript("""
CREATE TABLE Hospitals (
    hospital_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    address TEXT NOT NULL,
    contact_number TEXT NOT NULL,
    email TEXT NOT NULL,
    departments TEXT NOT NULL,
    opening_hours TEXT NOT NULL,
    rating REAL
);

CREATE TABLE Patients (
    patient_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    gender TEXT NOT NULL,
    contact_number TEXT NOT NULL,
    email TEXT NOT NULL,
    medical_history TEXT
);

CREATE TABLE Doctors (
    doctor_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    specialization TEXT NOT NULL,
    hospital_id INTEGER NOT NULL,
    experience_years INTEGER NOT NULL,
    contact_number TEXT NOT NULL,
    email TEXT NOT NULL,
    FOREIGN KEY (hospital_id) REFERENCES Hospitals(hospital_id)
);

CREATE TABLE Appointments (
    appointment_id TEXT PRIMARY KEY,
    patient_id TEXT NOT NULL,
    doctor_id INTEGER NOT NULL,
    department TEXT NOT NULL,
    date TEXT NOT NULL,
    time TEXT NOT NULL,
    status TEXT NOT NULL,
    FOREIGN KEY (patient_id) REFERENCES Patients(patient_id),
    FOREIGN KEY (doctor_id) REFERENCES Doctors(doctor_id)
);
""")

# Insert data into Hospitals
hospitals = load_json("hospitals.json")
for hospital in hospitals:
    cursor.execute("""
        INSERT INTO Hospitals (hospital_id, name, address, contact_number, email, departments, opening_hours, rating)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
        (hospital["hospital_id"], hospital["name"], hospital["address"], hospital["contact_number"],
         hospital["email"], json.dumps(hospital["departments"]), hospital["opening_hours"], hospital["rating"])
    )

# Insert data into Patients
patients = load_json("patients.json")
for patient in patients:
    cursor.execute("""
        INSERT INTO Patients (patient_id, name, age, gender, contact_number, email, medical_history)
        VALUES (?, ?, ?, ?, ?, ?, ?)""",
        (patient["patient_id"], patient["name"], patient["age"], patient["gender"],
         patient["contact_number"], patient["email"], json.dumps(patient["medical_history"]))  # Convert list to JSON string
    )

# Insert data into Doctors
doctors = load_json("doctors.json")
for doctor in doctors:
    cursor.execute("""
        INSERT INTO Doctors (doctor_id, name, specialization, hospital_id, experience_years, contact_number, email)
        VALUES (?, ?, ?, ?, ?, ?, ?)""",
        (doctor["doctor_id"], doctor["name"], doctor["specialization"], doctor["hospital_id"],
         doctor["experience_years"], doctor["contact_number"], doctor["email"])
    )

# Insert data into Appointments (validating department)
appointments = load_json("appointments.json")
for appointment in appointments:
    # Check if the doctor exists
    cursor.execute("SELECT specialization FROM Doctors WHERE doctor_id = ?", (appointment["doctor_id"],))
    doctor_specialization = cursor.fetchone()

    if doctor_specialization and doctor_specialization[0] == appointment["department"]:
        cursor.execute("""
            INSERT INTO Appointments (appointment_id, patient_id, doctor_id, department, date, time, status)
            VALUES (?, ?, ?, ?, ?, ?, ?)""",
            (appointment["appointment_id"], appointment["patient_id"], appointment["doctor_id"],
             appointment["department"], appointment["date"], appointment["time"], appointment["status"])
        )
    else:
        print(f"❌ Error: Doctor {appointment['doctor_id']} does not match department {appointment['department']} in appointment {appointment['appointment_id']}")

# Commit changes and close connection
conn.commit()
conn.close()

print("✅ Database created successfully!")


✅ Database created successfully!


In [ ]:
import sqlite3

conn = sqlite3.connect("hospital_db.sqlite3")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", [table[0] for table in tables])

conn.close()


Tables in the database: ['Hospitals', 'Patients', 'Doctors', 'Appointments']


In [ ]:
conn = sqlite3.connect("hospital_db.sqlite3")
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(Hospitals);")
columns = cursor.fetchall()
print("Doctors Table Schema:")
for col in columns:
    print(col)

conn.close()


Doctors Table Schema:
(0, 'hospital_id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'address', 'TEXT', 1, None, 0)
(3, 'contact_number', 'TEXT', 1, None, 0)
(4, 'email', 'TEXT', 1, None, 0)
(5, 'departments', 'TEXT', 1, None, 0)
(6, 'rating', 'REAL', 0, None, 0)
(7, 'weekdays_opening_time', 'TEXT', 0, None, 0)
(8, 'weekdays_closing_time', 'TEXT', 0, None, 0)
(9, 'saturday_opening_time', 'TEXT', 0, None, 0)
(10, 'saturday_closing_time', 'TEXT', 0, None, 0)
(11, 'sunday_opening_time', 'TEXT', 0, None, 0)
(12, 'sunday_closing_time', 'TEXT', 0, None, 0)


In [ ]:
conn = sqlite3.connect("hospital_db.sqlite3")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Hospitals LIMIT 5;")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()


(1, 'Green Valley Medical Center', '123 Elm Street, Springfield, IL', '+1-555-123-4567', 'info@greenvalleymedical.com', '["Cardiology", "Neurology", "Pediatrics", "Orthopedics", "Dermatology", "Oncology"]', 'Mon-Fri: 8 AM - 8 PM, Sat-Sun: 9 AM - 5 PM', 4.5)
(2, 'Sunrise General Hospital', '456 Oak Avenue, Denver, CO', '+1-555-234-5678', 'contact@sunrisehospital.com', '["Emergency", "Gynecology", "Urology", "Radiology", "Endocrinology", "Psychiatry"]', '24/7', 4.2)
(3, 'Lakeside Community Hospital', '789 Pine Road, Austin, TX', '+1-555-345-6789', 'support@lakesidehospital.com', '["Pediatrics", "Cardiology", "Orthopedics", "ENT", "Gastroenterology", "Nephrology"]', 'Mon-Sat: 7 AM - 9 PM, Sun: Closed', 4.7)
(4, 'Harmony Health Center', '321 Maple Lane, Seattle, WA', '+1-555-456-7890', 'hello@harmonyhealth.com', '["Dermatology", "Oncology", "Neurology", "Pulmonology", "Rheumatology", "Ophthalmology"]', 'Mon-Fri: 8 AM - 6 PM, Sat: 9 AM - 3 PM, Sun: Closed', 4.3)
(5, 'Evergreen Medical Insti

In [ ]:
import re

def convert_to_24hr(time_str):
    """Convert 12-hour format (e.g., 7 AM - 9 PM) to 24-hour format (07:00 - 21:00)."""
    time_str = time_str.replace("AM", " AM").replace("PM", " PM")  # Ensure space before AM/PM
    time_ranges = time_str.split(" - ")

    converted_times = []
    for time in time_ranges:
        match = re.match(r"(\d{1,2})\s*(AM|PM)", time.strip())
        if match:
            hour, period = int(match.group(1)), match.group(2)
            if period == "PM" and hour != 12:
                hour += 12
            if period == "AM" and hour == 12:
                hour = 0
            converted_times.append(f"{hour:02d}:00")

    return " - ".join(converted_times) if len(converted_times) == 2 else "Closed"

def parse_opening_hours(opening_hours):
    """Extract and format weekdays, Saturday, and Sunday hours."""
    if "24/7" in opening_hours:
        return "00:00 - 23:59", "00:00 - 23:59", "00:00 - 23:59"

    weekdays_match = re.search(r"(Mon-Fri|Mon-Sat|Mon-Sun):\s*([\d\sAMP-]+)", opening_hours)
    saturday_match = re.search(r"Sat:\s*([\d\sAMP-]+)", opening_hours)
    sunday_match = re.search(r"Sun:\s*([\d\sAMP-]+)", opening_hours)

    weekdays = convert_to_24hr(weekdays_match.group(2)) if weekdays_match else "Closed"
    saturday = convert_to_24hr(saturday_match.group(1)) if saturday_match else "Closed"
    sunday = convert_to_24hr(sunday_match.group(1)) if sunday_match else "Closed"

    return weekdays, saturday, sunday
